# Imports and Downloads

In [ ]:
# A
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords 
stop_words = set(stopwords.words('english'))
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer 
wordnet_lemmatizer = WordNetLemmatizer() 


import pandas as pd
import os
import pickle
import numpy as np
from bs4 import BeautifulSoup
import string
import re
import random
import math
import sys

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.metrics import calinski_harabasz_score
from collections import Counter

import gensim.downloader as api
import tensorflow_hub as hub

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
# B
from google.colab import drive
drive.mount('/content/drive')

# Files Reading, Pre-Processing & Saving for Training

In [ ]:
# C
!unzip './drive/MyDrive/DatasetsZip/webkb'
!unzip './drive/MyDrive/DatasetsZip/20news-18828'
!unzip './drive/MyDrive/DatasetsZip/reuters'
!unzip './drive/MyDrive/DatasetsZip/news Ag'
!unzip './drive/MyDrive/DatasetsZip/bbc news'
!unzip './bbc news/BBC News Test.csv'
!unzip './bbc news/BBC News Train.csv'
!unzip './drive/MyDrive/DatasetsZip/webkb2'

unzip:  cannot find or open ./drive/MyDrive/DatasetsZip/reuters, ./drive/MyDrive/DatasetsZip/reuters.zip or ./drive/MyDrive/DatasetsZip/reuters.ZIP.


In [ ]:
# WebKB Dataset

webkbData = []

path = './webkb'
for r, d, f in os.walk(path):

  for file in f:
      file_temp = ""
      p = os.path.join(r, file)
      file_temp = open(p,'r', errors='ignore').read()
      soup = BeautifulSoup(file_temp, 'html.parser')
      body = soup.text
      body = str(body)
      # body = str(np.char.lower(body))
      webkbData.append(body)

In [ ]:
# News20 Dataset

news20Data = []

path = './20news-18828'
for r, d, f in os.walk(path):

  for file in f:
      file_temp = ""
      p = os.path.join(r, file)
      file_temp = open(p,'r', errors='ignore').read()
      body = str(file_temp)
      # body = str(np.char.lower(file_temp))
      news20Data.append(body)

In [ ]:
# Reuters Dataset

reutersData = []

path = './reuters'
for r, d, f in os.walk(path):

  for file in f:
      file_temp = ""
      p = os.path.join(r, file)
      file_temp = open(p,'r', errors='ignore').read()
      body = str(file_temp)
      # body = str(np.char.lower(file_temp))
      reutersData.append(body)

In [ ]:
# AG-News Dataset

train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')

train['content'] = train['Title']+' '+train['Description']
test['content'] = test['Title']+' '+test['Description']

ag_newsData = list(train['content']) + list(test['content'])

In [ ]:
# BBC-News Dataset

train = pd.read_csv('./BBC News Train.csv')
test = pd.read_csv('./BBC News Test.csv')

bbc_newsData = list(train['Text']) + list(test['Text'])

In [ ]:
rawDataset = webkbData + news20Data + reutersData + ag_newsData + bbc_newsData

In [ ]:
len(rawDataset)

167726

In [ ]:
# Pre-processing

# test_list = ['Date: ', 'Server: ', 'Content-type: ', 'Content-length: ', 'Last-modified: ', 'From: ', 'Subject: '] 
test_list = [] 
# remove punctuation from each word
table = str.maketrans('', '', string.punctuation)
Dataset=[]
for doc in rawDataset: 
  lines = list(filter(bool, doc.splitlines()))
  temp_line=""
  for line in lines:
    if not any(map(line.__contains__, test_list)):
      # convert to lower case
      line = str(np.char.lower(line))
      line = re.sub(r'\d+', '', line)
      # text_tokens = word_tokenize(line)
      text_tokens = re.split(r'\W+', line)
      for word in text_tokens: 
          # remove remaining tokens that are not alphabetic and filter out sopwords
          if ((word not in stop_words) and (word.isalpha())): 
              # temp_line = temp_line + " " + wordnet_lemmatizer.lemmatize(word)
              temp_line = temp_line + " " + word.translate(table)
  Dataset.append(temp_line.strip()) 

In [ ]:
rawDataset[0]

"Date: Tue, 10 Dec 1996 17:53:27 GMT\nServer: NCSA/1.4.2\nContent-type: text/html\nLast-modified: Fri, 07 Jun 1996 23:08:01 GMT\nContent-length: 323\n\n\nShopBot\n\nThe ShopBot Has Moved\n\nIf your browser supports Java, please use the \nnew version here.\n\nIf your browser doesn't support Java, you'll have to resort to the \nold out-of-date version here.\n\n\n"

In [ ]:
Dataset[0]

'date tue dec gmt server ncsa content type text html last modified fri jun gmt content length shopbot shopbot moved browser supports java please use new version browser support java resort old date version'

# CBOW

In [ ]:
word2vec_model = api.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [ ]:
print(word2vec_model.most_similar("computer"))

In [ ]:
print(len(word2vec_model["cat"]))

# GloVe

In [ ]:
glove_model = api.load('glove-wiki-gigaword-300')

[==================================================] 100.0% 376.1/376.1MB downloaded


In [ ]:
print(glove_model.most_similar("computer"))

[('computers', 0.8248153924942017),
 ('software', 0.733441948890686),
 ('pc', 0.6240140199661255),
 ('technology', 0.6198545694351196),
 ('computing', 0.6178765296936035),
 ('laptop', 0.5955508947372437),
 ('internet', 0.5857782363891602),
 ('ibm', 0.5825320482254028),
 ('systems', 0.574499249458313),
 ('hardware', 0.5728795528411865)]

In [ ]:
print(len(glove_model["cat"]))

300

# Universal Sentence Encoder

In [ ]:
u_s_e_model = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
print(len(u_s_e_model(["computer"])[0]))

# FastText

In [ ]:
# D
!git clone https://github.com/facebookresearch/fastText.git

fatal: destination path 'fastText' already exists and is not an empty directory.


In [ ]:
# E
cd fastText

/content/fastText


In [ ]:
# F
!sudo pip install .

Processing /content/fastText
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3086290 sha256=4fa731f326f607bb6760ced9256df73147b968ca865906e6fd8afd82d3185898
  Stored in directory: /tmp/pip-ephem-wheel-cache-ms1f32rj/wheels/a1/9f/52/696ce6c5c46325e840c76614ee5051458c0df10306987e7443
Successfully built fasttext
  Found existing installation: fasttext 0.9.2
    Uninstalling fasttext-0.9.2:
      Successfully uninstalled fasttext-0.9.2


In [ ]:
f = open("train.txt", "a")
for txt in Dataset:
  f.write(txt+"\n")
f.close()

In [ ]:
import fasttext
fasttext_model = fasttext.train_unsupervised(input='train.txt', dim=300, model='skipgram')

In [ ]:
fasttext_model.save_model("train.bin")

In [ ]:
print(fasttext_model.get_nearest_neighbors("green",20))

In [ ]:
print(len(fasttext_model['green']))

300

In [ ]:
# G
cd ..

/content


# Files Reading, Pre-Processing and Calculation of TF-IDF

In [ ]:
# Extracting data

path = './webkb2'
files = []
labels = []
for r, d, f in os.walk(path):

  for file in f:
      file_temp = ""
      label = r.rsplit('/', 1)[1]
      p = os.path.join(r, file)
      file_temp = open(p,'r', errors='ignore').read()
      soup = BeautifulSoup(file_temp, 'html.parser')
      body = soup.text
      body = str(np.char.lower(body))

      files.append(body)
      labels.append(label)

In [ ]:
print(set(labels))

{'cornell', 'texas', 'washington', 'wisconsin'}

In [ ]:
print(files[0])

'date: thu, 07 nov 1996 19:16:49 gmt\nserver: ncsa/1.5\ncontent-type: text/html\nlast-modified: fri, 11 oct 1996 18:05:48 gmt\ncontent-length: 2322\n\n\n\ntodd turnidge\n\n\n\n\ntodd douglas turnidge\n\nschool\n\ncomputer sciences department\nuniversity of wisconsin - madison\n1210 w. dayton st.\nmadison, wi  53706\n(608) 262-6612\n\nhome\n\n\n1124 milton st.\nmadison, wi  53715\n(608) 250-0699\n\n\n\ni am a graduate student in the \ndepartment of computer sciences\nat the university of wisconsin,\nmadison.  i have been here for two years.  i am working with professor\nthomas reps studying\nprogramming languages.\ni teach a section of cs302.\ni hold a bs in mathematics\nand an ms in computer\nscience from case western reserve\nuniversity, which is located in  cleveland,\nohio.\ni am originally from kent, ohio.  my\nfamily lives there.\n\nthey say you can judge a man by the company he keeps.  click here for enough evidence to put me away for a long time.\nsome amusements for you.\nsome 

In [ ]:
# Pre-processing

stemFeatures = {}
# test_list = ['Date: ', 'Server: ', 'Content-type: ', 'Content-length: ', 'Last-modified: ', 'From: ', 'Subject: '] 
test_list = [] 
# remove punctuation from each word
table = str.maketrans('', '', string.punctuation)
preprocessed_lines=[]
for doc in files: 
  lines = list(filter(bool, doc.splitlines()))
  temp_line=""
  for line in lines:
    if not any(map(line.__contains__, test_list)):
      # convert to lower case
      line = str(np.char.lower(line))
      line = re.sub(r'\d+', '', line)
      # text_tokens = word_tokenize(line)
      text_tokens = re.sub("-", "", line)
      text_tokens = re.split(r'\W+', line)
      for word in text_tokens: 
          # remove remaining tokens that are not alphabetic and filter out sopwords
          if ((word not in stop_words) and (word.isalpha())): 
              word = wordnet_lemmatizer.lemmatize(word)
              # stemFeatures[word] = porter.stem(word)
              # word = porter.stem(word)
              temp_line = temp_line + " " + word.translate(table)
  preprocessed_lines.append(temp_line.strip()) 

files = preprocessed_lines

In [ ]:
print(files[0])

'date thu nov gmt server ncsa content type text html last modified fri oct gmt content length todd turnidge todd douglas turnidge school computer science department university wisconsin madison w dayton st madison wi home milton st madison wi graduate student department computer science university wisconsin madison two year working professor thomas rep studying programming language teach section c hold b mathematics m computer science case western reserve university located cleveland ohio originally kent ohio family life say judge man company keep click enough evidence put away long time amusement shortcut last modified fri oct todd turnidge turnidge c wisc edu'

In [ ]:
# tf–idf

vectorizer = TfidfVectorizer(stop_words='english', sublinear_tf=True, strip_accents='unicode',
    analyzer='word', token_pattern=r'\w{2,}', 
    ngram_range=(1, 1), max_features=100).fit(files)    # no. of features
X = vectorizer.fit_transform(files)
features = vectorizer.get_feature_names()

In [ ]:
print(features)

['acm', 'address', 'algorithm', 'analysis', 'application', 'architecture', 'assignment', 'austin', 'available', 'based', 'cern', 'class', 'computer', 'computing', 'conference', 'content', 'cornell', 'course', 'cse', 'data', 'database', 'date', 'department', 'design', 'distributed', 'edu', 'email', 'engineering', 'exam', 'fall', 'file', 'gmt', 'graduate', 'group', 'home', 'homework', 'hour', 'html', 'image', 'information', 'jan', 'language', 'lecture', 'length', 'link', 'list', 'machine', 'madison', 'mail', 'memory', 'mime', 'modified', 'monday', 'ncsa', 'network', 'new', 'note', 'nov', 'object', 'oct', 'office', 'operating', 'page', 'paper', 'parallel', 'performance', 'phone', 'postscript', 'problem', 'proceeding', 'program', 'programming', 'project', 'publication', 'report', 'research', 'science', 'section', 'server', 'software', 'solution', 'student', 'texas', 'text', 'th', 'thu', 'time', 'tuesday', 'type', 'university', 'use', 'using', 'utexas', 'version', 'washington', 'web', 'wedn

In [ ]:
# H
def purity(labels, clustered):
    
    cluster_ids = set(clustered)

    N = len(clustered)
    majority_sum = 0  
    for cl in cluster_ids:
        labels_cl = Counter(l for l, c in zip(labels, clustered) if c == cl)
        majority_sum += max(labels_cl.values())

    return majority_sum / N

In [ ]:
dense = X.todense()
denselist = dense.tolist()

In [ ]:
# Tf-Idf Purity Calculation

kmeans = KMeans(n_clusters=4, random_state=0)
clustered_docs = kmeans.fit_predict(X)


print(purity(labels, clustered_docs))

# Expand with Embedding Models

In [ ]:
# Replacing every feature with its fasttext representation

fasttext_docs = []
fasttext_dist = []

for file in denselist:
  dist = 0
  counter = 0
  long_vector = []
  for feature in features:
      if(file[counter] == 0.0):   # feature not in doc
          long_vector.append(0.0)
          counter+=1
      else:
          try:
              long_vector.append(file[counter]*(sum(fasttext_model[feature]) / 300))   # feature representation available
          except:
              long_vector.append(file[counter])   # feature representation not available
          counter+=1
  dist = math.sqrt(np.sum(np.power((np.array(file)-np.array(long_vector)),2)))
  long_vector = np.array(long_vector) * dist
  fasttext_dist.append(dist)
  fasttext_docs.append(long_vector)

In [ ]:
# Replacing every feature with its word2vec representation

word2vec_docs = []
word2vec_dist = []

for file in denselist:
  dist = 0
  counter = 0
  long_vector = []
  for feature in features:
      if(file[counter] == 0.0):   # feature not in doc
          long_vector.append(0.0)
          counter+=1
      else:
          try:
              long_vector.append(file[counter]*(sum(word2vec_model[feature]) / 300))   # feature representation available
          except:
              long_vector.append(file[counter])   # feature representation not available
          counter+=1
  dist = math.sqrt(np.sum(np.power((np.array(file)-np.array(long_vector)),2)))
  long_vector = np.array(long_vector) * dist
  word2vec_dist.append(dist)
  word2vec_docs.append(long_vector)

In [ ]:
# Replacing every feature with its glove representation

glove_docs = []
glove_dist = []

for file in denselist:
  dist = 0
  counter = 0
  long_vector = []
  for feature in features:
      if(file[counter] == 0.0):   # feature not in doc
          long_vector.append(0.0)
          counter+=1
      else:
          try:
              long_vector.append(file[counter]*(sum(glove_model[feature]) / 300))   # feature representation available
          except:
              long_vector.append(file[counter])   # feature representation not available
          counter+=1
  dist = math.sqrt(np.sum(np.power((np.array(file)-np.array(long_vector)),2)))
  long_vector = np.array(long_vector) * dist
  glove_dist.append(dist)
  glove_docs.append(long_vector)

In [ ]:
# Replacing every feature with its u_s_e_ representation

u_s_e_docs = []
u_s_e_dist = []

for file in denselist:
  dist = 0
  counter = 0
  long_vector = []
  for feature in features:
      if(file[counter] == 0.0):   # feature not in doc
          long_vector.append(0.0)
          counter+=1
      else:
          try:
              long_vector.append(file[counter]*(sum(u_s_e_model([feature])[0]) / 512))   # feature representation available
          except:
              long_vector.append(file[counter])   # feature representation not available
          counter+=1
  dist = math.sqrt(np.sum(np.power((np.array(file)-np.array(long_vector)),2)))
  long_vector = np.array(long_vector) * dist
  u_s_e_dist.append(dist)
  u_s_e_docs.append(long_vector)

# Docs Conversion, Clustering and Evaluating

In [ ]:
# Multiplying docs with distance coefficient from raw doc
total = np.array(fasttext_dist) + np.array(word2vec_dist) + np.array(glove_dist) + np.array(u_s_e_dist)
final_docs = []

for i in range(len(glove_docs)):
  total[i] = 1.0 if total[i] == 0.0 else total[i]
  fasttext_docs[i] = np.array(fasttext_docs[i]) / total[i]
  word2vec_docs[i] = np.array(word2vec_docs[i]) / total[i]
  glove_docs[i] = np.array(glove_docs[i]) / total[i]
  u_s_e_docs[i] = np.array(u_s_e_docs[i]) / total[i]

  final_docs.append(np.array((fasttext_docs[i] + word2vec_docs[i] + glove_docs[i] + u_s_e_docs[i]) / 2))

In [ ]:
# Conversion to unit vector
for i in range(len(final_docs)):
  magnitude = np.linalg.norm(np.array(final_docs[i]))
  magnitude = 1.0 if magnitude == 0.0 else magnitude
  final_docs[i] = np.array(final_docs[i]) / magnitude

In [ ]:
# K-Means Algorithm

clustered_docs = []
# I
def kmeans(s1, s2, s3, s4, docs):
  clusters = [[], [], [], []]

  for i in range(len(docs)):

    d1 = 0
    d2 = 0
    d3 = 0
    d4 = 0
  
    d1 = math.sqrt(np.sum(np.power((np.array(s1)-np.array(docs[i])),2)))
    d2 = math.sqrt(np.sum(np.power((np.array(s2)-np.array(docs[i])),2)))
    d3 = math.sqrt(np.sum(np.power((np.array(s3)-np.array(docs[i])),2)))
    d4 = math.sqrt(np.sum(np.power((np.array(s4)-np.array(docs[i])),2)))

    mini = min(d1, d2, d3, d4)
    if(mini == d1):
      clusters[0].append(docs[i])
      clustered_docs.append(0)
    elif(mini == d2):
      clusters[1].append(docs[i])
      clustered_docs.append(1)
    elif(mini == d3):
      clusters[2].append(docs[i])
      clustered_docs.append(2)
    elif(mini == d4):
      clusters[3].append(docs[i])
      clustered_docs.append(3)
      
  return clusters

In [ ]:
# Calculating Inter and Intra Cluster Similarity on every iteration

rand = random.sample(range(0, len(files)), 4)
seeds = []
seeds.append(final_docs[rand[0]])
seeds.append(final_docs[rand[1]])
seeds.append(final_docs[rand[2]])
seeds.append(final_docs[rand[3]])

for i in range(30):
  clustered_docs = []
  clusters = kmeans(seeds[0], seeds[1], seeds[2], seeds[3], final_docs)
  seeds[0] = np.array([0]*100)
  seeds[1] = np.array([0]*100)
  seeds[2] = np.array([0]*100)
  seeds[3] = np.array([0]*100)
  
  for doc in clusters[0]:
    seeds[0] = np.array(seeds[0]) + np.array(doc)
  seeds[0] /= len(clusters[0])

  for doc in clusters[1]:
   seeds[1] = np.array(seeds[1]) + np.array(doc)
  seeds[1] /= len(clusters[1])

  for doc in clusters[2]:
   seeds[2] = np.array(seeds[2]) + np.array(doc)
  seeds[2] /= len(clusters[2])

  for doc in clusters[3]:
   seeds[3] = np.array(seeds[3]) + np.array(doc)
  seeds[3] /= len(clusters[3])

  print("Purity :", purity(labels, clustered_docs))

  silhouette_avg = silhouette_score(final_docs, np.array(clustered_docs))
  print("The average silhouette_score is :", silhouette_avg)

  calinski_harabasz_avg = calinski_harabasz_score(final_docs, np.array(clustered_docs))
  print("The average calinski_harabasz_score is :", calinski_harabasz_avg)
  print("*********************************")

Purity : 0.5609532538955087
The average silhouette_score is : 0.06979149307570438
The average calinski_harabasz_score is : 195.53914429702326
*********************************
Purity : 0.8625114573785518
The average silhouette_score is : 0.5850651312232006
The average calinski_harabasz_score is : 791.0590833184986
*********************************
Purity : 0.8661778185151238
The average silhouette_score is : 0.672798505343015
The average calinski_harabasz_score is : 1294.4585019256763
*********************************
Purity : 0.8661778185151238
The average silhouette_score is : 0.672798505343015
The average calinski_harabasz_score is : 1294.4585019256763
*********************************
Purity : 0.8661778185151238
The average silhouette_score is : 0.672798505343015
The average calinski_harabasz_score is : 1294.4585019256763
*********************************
Purity : 0.8661778185151238
The average silhouette_score is : 0.672798505343015
The average calinski_harabasz_score is : 1294.45